In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my-test-project-466009-b10d519c6fd7.json"

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import mlflow
import mlflow.keras
from mlflow.tracking import MlflowClient

# Prepare mlflow tracking URI (file-based)
tracking_dir = os.path.join(os.getcwd(), "mlruns")
os.makedirs(tracking_dir, exist_ok=True)
tracking_dir_fixed = tracking_dir.replace("\\", "/")
tracking_uri = f"file:///{tracking_dir_fixed}"
print(f"🔗 MLflow Tracking URI: {tracking_uri}")

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("cat-vs-dog-hyperparam-tuning")

# Enable autolog (it will log many Keras parameters/metrics automatically)
mlflow.keras.autolog()

# Use a single variable for epochs so it's consistent everywhere
EPOCHS = 5

with mlflow.start_run(run_name="mobilenetv2-cat-vs-dog"):
    # Load pretrained MobileNetV2 as base model
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(64, 64, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze pretrained layers

    # Build sequential model on top of base model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(
        optimizer=optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Data generators
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    training_set = train_datagen.flow_from_directory(
        r"C:\Users\pares\OneDrive\Documents\vs code\are-u-cat\dataset\training_set",
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
    )

    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_set = test_datagen.flow_from_directory(
        r"C:\Users\pares\OneDrive\Documents\vs code\are-u-cat\dataset\test_set",
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
    )

    # Train
    history = model.fit(training_set, validation_data=test_set, epochs=EPOCHS)

    # --- IMPORTANT: avoid re-logging keys autolog already wrote ---
    # Option: store custom metadata as tags (safe)
    mlflow.set_tag("manual_input_shape", "64x64x3")
    mlflow.set_tag("manual_optimizer", "Adam")
    mlflow.set_tag("manual_epochs", str(EPOCHS))

    # If you do want params, use unique keys to avoid collision:
    mlflow.log_param("manual_batch_size", 32)

    # Log final metrics manually if you like (autolog usually already logs epoch metrics)
    mlflow.log_metric("final_train_accuracy", history.history['accuracy'][-1])
    mlflow.log_metric("final_val_accuracy", history.history['val_accuracy'][-1])

    # Explicitly log the model (autolog may already have done this)
    mlflow.keras.log_model(model, "model")

    print(f"✅ Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print("🎯 MLflow run completed successfully!")

    # Test single image
    test_image = load_img(
        r"C:\Users\pares\OneDrive\Documents\vs code\are-u-cat\dataset\test_set\dogs\dog.4025.jpg",
        target_size=(64, 64)
    )
    test_image = img_to_array(test_image) / 255.0
    test_image = np.expand_dims(test_image, axis=0)

    result = model.predict(test_image)
    prediction = 'dog' if result[0][0] >= 0.5 else 'cat'
    print("Prediction:", prediction)

# Inspect registered models (optional)
client = MlflowClient()
for rm in client.search_registered_models():
    print(rm.name)

# If you want to switch to a remote tracking server:
# mlflow.set_tracking_uri("http://104.197.55.178:5000/")
# mlflow.set_experiment("cat-vs-dog-hyperparam-tuning")


Traceback (most recent call last):
  File "C:\Users\pares\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pares\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pares\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retrie

🔗 MLflow Tracking URI: file:///c:/Users/pares/OneDrive/Documents/vs code/are-u-cat/mlruns


C:\Users\pares\AppData\Local\Temp\ipykernel_192\2313524773.py:30: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


Found 7998 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.


2025/09/23 21:03:33 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2025/09/23 21:03:33 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
C:\Users\pares\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 78ms/step - accuracy: 0.7803 - loss: 0.4664 - val_accuracy: 0.8152 - val_loss: 0.3997
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.8417 - loss: 0.3447 - val_accuracy: 0.8162 - val_loss: 0.4003
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 67ms/step - accuracy: 0.8730 - loss: 0.2881 - val_accuracy: 0.8247 - val_loss: 0.4062
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 48s 192ms/step - accuracy: 0.9034 - loss: 0.2320 - val_accuracy: 0.8132 - val_loss: 0.4339
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.9297 - loss: 0.1793 - val_accuracy: 0.8097 - val_loss: 0.4623


2025/09/23 21:06:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/23 21:06:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/23 21:06:24 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/09/23 21:06:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Final Validation Accuracy: 0.8097
🎯 MLflow run completed successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: dog
aaaaaaa
are- u-a-cat-or-dog2
are-u-a-cat-or-dog


In [2]:
model.save("cat_dog_model.h5")
